In [ ]:
import cv2
import webcolors

# Define a função que retorna o nome da cor RGB mais próxima
def get_color_name(rgb):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb[0]) ** 2
        gd = (g_c - rgb[1]) ** 2
        bd = (b_c - rgb[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

# Captura o vídeo da webcam
cap = cv2.VideoCapture(0)

while True:
    # Lê um frame do vídeo
    ret, frame = cap.read()

    # Obtém as dimensões do frame
    height, width, _ = frame.shape

    # Define o retângulo que delimita a região central do frame
    x1 = int(0.25 * width)
    y1 = int(0.25 * height)
    x2 = int(0.75 * width)
    y2 = int(0.75 * height)

    # Segmenta a região central do frame
    roi = frame[y1:y2, x1:x2]

    # Obtém a cor média da região central do frame
    bgr_color = cv2.mean(roi)[:3]
    rgb_color = tuple(reversed(bgr_color))
    color_name = get_color_name(rgb_color)

    # Escreve o nome da cor na imagem
    cv2.putText(frame, color_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Exibe a imagem resultante
    cv2.imshow('frame', frame)

    # Espera pela tecla 'q' para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera a captura de vídeo e fecha a janela
cap.release()
cv2.destroyAllWindows()



In [ ]:
import cv2
import webcolors

# Define a função que retorna o nome da cor RGB mais próxima
def get_color_name(rgb):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb[0]) ** 2
        gd = (g_c - rgb[1]) ** 2
        bd = (b_c - rgb[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

# Captura o vídeo da webcam
cap = cv2.VideoCapture(0)

while True:
    # Lê um frame do vídeo
    ret, frame = cap.read()

    # Obtém as dimensões do frame
    height, width, _ = frame.shape

    # Define o retângulo que delimita a região central do frame
    x1 = int(0.25 * width)
    y1 = int(0.25 * height)
    x2 = int(0.75 * width)
    y2 = int(0.75 * height)

    # Segmenta a região central do frame
    roi = frame[y1:y2, x1:x2]

    # Obtém a cor média da região central do frame
    bgr_color = cv2.mean(roi)[:3]
    rgb_color = tuple(reversed(bgr_color))
    color_name = get_color_name(rgb_color)

    # Desenha um retângulo ao redor da região central do frame
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Escreve o nome da cor em português dentro do retângulo
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f'Cor: {color_name}'
    org = (x1 + 10, y1 + 30)
    fontScale = 0.8
    color = (255, 255, 255)
    thickness = 2
    cv2.putText(frame, text, org, font, fontScale, color, thickness)

    # Exibe o vídeo resultante
    cv2.imshow('frame', frame)

    # Espera pela tecla 'q' para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera a captura de vídeo e fecha a janela
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import dlib
import numpy as np
from keras.models import load_model

# Carrega o modelo pré-treinado para detecção de pontos faciais
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Carrega o modelo pré-treinado para classificação de emoções
model = load_model('model.h5')

# Define um dicionário para mapear os rótulos de saída do modelo para emoções
emotions = {
    0: "Raiva",
    1: "Desgosto",
    2: "Medo",
    3: "Feliz",
    4: "Triste",
    5: "Surpresa",
    6: "Neutro"
}

# Inicializa a webcam
cap = cv2.VideoCapture(0)

while True:
    # Captura o frame da webcam
    ret, frame = cap.read()

    # Redimensiona o frame para melhor desempenho
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Converte o frame para escala de cinza
    gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)

    # Detecta os pontos faciais no frame
    faces = detector(gray)

    # Itera sobre cada rosto detectado
    for face in faces:
        # Obtém os pontos faciais do rosto
        landmarks = predictor(gray, face)

        # Converte os pontos faciais para um vetor NumPy
        landmarks = np.array([(p.x, p.y) for p in landmarks.parts()])

        # Redimensiona os pontos faciais para o tamanho original do frame
        landmarks *= 4

        # Extrai a região de interesse correspondente ao rosto
        x1, y1 = np.min(landmarks, axis=0)
        x2, y2 = np.max(landmarks, axis=0)
        roi = gray[y1:y2, x1:x2]

        # Redimensiona a região de interesse para o tamanho esperado pelo modelo
        roi = cv2.resize(roi, (48, 48))

        # Normaliza a região de interesse
        roi = roi.astype("float") / 255.0

        # Adiciona uma dimensão extra ao tensor de entrada do modelo
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        # Classifica a emoção da região de interesse
        predictions = model.predict(roi)
        label = emotions[np.argmax(predictions)]

        # Desenha um retângulo em torno do rosto
        cv2.rectangle(frame, (x1*4, y1*4), (x2*4, y2*4), (0, 255, 0), 2)

        # Escreve o rótulo da emoção sobre o retângulo
